In [5]:
from decouple import config
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [6]:
OPEN_AI_API_KEY = config('OPEN_AI_KEY')
UPSTASH_VECTOR_URL = config('UPSTASH_VECTOR_ENDPOINT')
UPSTASH_VECTOR_TOKEN = config('UPSTASH_VECTOR_TOKEN')

In [7]:
from openai import OpenAI

client = OpenAI(api_key=OPEN_AI_API_KEY)

In [8]:
from upstash_vector import Vector
from upstash_vector import Index

index = Index(url=UPSTASH_VECTOR_URL,token=UPSTASH_VECTOR_TOKEN)

In [11]:
# OPENAI EMBEDDING METHOD 
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [12]:
documents = [
    "Blue elephants dance at midnight.",
    "Quantum physics makes my coffee taste better.",
    "Seven ducks wearing sunglasses walked into a bar.",
    "The sky whispered a lullaby to the restless stars.",
    "Never trust a penguin with a briefcase."
    "The cat danced on the rooftop under the full moon.",
    "She found a mysterious key hidden under the old carpet.",
    "A sudden gust of wind knocked over the stack of books.",
    "He couldn't believe the spaceship was actually real.",
    "The smell of fresh coffee filled the entire house.",
    "Lightning flashed across the dark and stormy sky.",
    "A talking parrot revealed the secret treasure map.",
    "The clock struck midnight, and everything changed.",
    "She laughed as the puppy chased its own tail.",
    "An ancient book contained secrets lost to time.",
    "The robot blinked twice before speaking in perfect English.",
    "A secret door appeared behind the bookshelf.",
    "The ice cream truck played an eerie tune as it drove by.",
    "He accidentally discovered a portal to another dimension.",
    "The detective found a cryptic note under the suspect’s chair.",
    "A butterfly landed on her hand and whispered a message.",
    "The abandoned house had lights flickering in the attic.",
    "A fisherman caught a golden fish that granted three wishes.",
    "She woke up to find a tiny dragon sleeping on her pillow.",
    "A mysterious package arrived with no return address.",
    "The magician’s hat was deeper than it seemed.",
    "A single rose bloomed in the middle of the desert.",
    "The haunted mirror reflected a different reality.",
    "A hidden tunnel led to an underground kingdom.",
    "His shadow moved even when he stood still.",
    "The elevator button labeled 'Mystery' took them somewhere unexpected.",
    "She received a letter from her future self.",
    "The moon changed colors as the night progressed.",
    "A treehouse in the woods held a long-lost secret.",
    "The music box played a tune no one recognized.",
    "A scientist accidentally created a new species of plant.",
    "A painting in the museum blinked at her.",
    "The river flowed uphill for exactly one hour every day.",
    "A clock in the town square started ticking backward.",
    "The clouds formed shapes that told a story.",
    "She stepped into a puddle and ended up in another world.",
    "A library book contained handwritten notes from a ghost.",
    "The wind carried whispers of an unknown language.",
    "An old camera revealed things invisible to the human eye.",
    "A lighthouse beamed a message in Morse code.",
    "The statue in the park moved when no one was looking.",
    "A mysterious melody played from an abandoned piano.",
    "His reflection in the mirror didn’t match his movements.",
    "The town vanished from the map overnight.",
    "A pair of sunglasses let her see people’s true emotions.",
    "The vending machine dispensed something that wasn’t on the menu.",
    "Every midnight, the streetlights flickered in perfect sync.",
    "A message in a bottle arrived from the past.",
    "The old radio picked up signals from another galaxy.",
    "A snow globe contained a miniature version of their town."
]

In [13]:
embeddings = [get_embedding(x) for x in documents]

In [16]:
# convert list into a key:value pairs

dataset = {}

for i, embeddings in enumerate(embeddings):
    dataset[i] = embeddings

In [18]:
vectors = []
for key, value in dataset.items():
    id = key
    embedding = value

    vectors.append(Vector(id=id,vector=embedding))

In [20]:
# insert vector to Upstash index

index.upsert(
    vectors=vectors
)

'Success'

In [21]:
# query from Upstash index
query_str = "The Flash woke up to find a tiny dragon sleeping on her pillow."
query_embedded = get_embedding(query_str)

In [23]:
results = index.query(
    vector=query_embedded,
    top_k=5,
    include_vectors=True,
    include_metadata=True,
)

In [24]:
for result in results: 
    print(result.id, result.score)

22 0.9125993
19 0.6798767
35 0.65760505
9 0.6551604
5 0.65111583


In [ ]:
import numpy as np

In [ ]:
def calculate_cosine_metrics(v1,v2):
    dot_product = np.dot(v1,v2)
    magnitude1 = np.linalg.norm(v1)
    magnitude2 = np.linalg.norm(v2)
    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    cosine_distance = 1 - cosine_similarity

    return int(cosine_similarity * 100), int(cosine_distance * 100)

In [ ]:
query_str = "I am the Flash!"
query_embedded = get_embedding(query_str)

for emb in embeddings:
    print(calculate_cosine_metrics(query_embedded,emb))